In [67]:
#Load necessary packages
import pandas as pd
import os
import numpy as np
import copy

import matplotlib.pylab as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

from numpy.polynomial.polynomial import polyfit

### Data Prep - all code condensed from Library_Week2_Notebook4_Strategy#2

In [3]:
##CODE FROM NOTEBOOK: Library_Week2_Notebook4_Strategy#2
#(Load and process data created in Library_Week2_Notebook4_Strategy#2)

#Load data
os.chdir("//Users/Olga/Documents/INSIGHT2019/Library data/AllPldData")
Files_in_folder = os.listdir()
File_names = ['_2014', '_2015', '_2016', '_2001', '_2002', '_2003', 
              '_2004','_2005', '_2006', '_2007', '_2008', '_2009', 
              '_2010', '_2011', '_2012', '_2013', '_2000', '_1992', 
              '_1993', '_1994', '_1995', '_1996', '_1997', '_1998', '_1999']
Files = []

for filename in os.listdir():
    if filename.endswith('csv'):
        Files.append(pd.read_csv(filename, encoding = 'latin-1', low_memory = False))
        
#create dictionary of libraries labeled by year
LibData_dict = {}
for i in range(0, len(File_names)):
    LibData_dict[File_names[i]] = Files[i]
    
for k, v in LibData_dict.items():
    print(k, v.shape, type(LibData_dict[k]))

ColumnsPresent = pd.DataFrame(index=range(0),columns=range(0))

for k, v in LibData_dict.items():
    dataframe_to_join = pd.DataFrame({k: list(v.columns)})
    ColumnsPresent = ColumnsPresent.join(dataframe_to_join, how = 'outer')

#Rearrange columns in dataframe. 
cols = ColumnsPresent.columns.tolist()
cols_rearr = cols[17:] + cols[0:17]
cols_rearr2 = cols_rearr[0:8] + cols_rearr[24:25] + cols_rearr[11:24] + cols_rearr[8:11]

ColumnsPresent_rearr = ColumnsPresent[cols_rearr2]

ColumnsPresent_rearr_str = ColumnsPresent_rearr.applymap(str)
ColumnsPresent_rearr_str = ColumnsPresent_rearr_str.reset_index(drop = True)

#create DF of unique column values
UniqueLabels = np.unique(ColumnsPresent_rearr_str.values)
LabelsBoolDF = pd.DataFrame(UniqueLabels)

#Create DF w boolean values for prensence of each unique value
#THIS IS THE END GOAL OF THIS DATA
for j in range(len(ColumnsPresent_rearr_str.columns)):
    lis = []
    for i in range(len(UniqueLabels)):
        lis.append(ColumnsPresent_rearr_str.iloc[:, j].isin([UniqueLabels[i]]).any())
    LabelsBoolDF[ColumnsPresent_rearr_str.columns[j]] = lis
    
## Sum all TRUE values. Value of 25 = all true
sum_list = []
for i in range(LabelsBoolDF.shape[0]):
    sum_list.append(sum(LabelsBoolDF.iloc[i, 1:26]))

#Label OK in Notes
LabelsBoolDF['TRUE SUM'] = sum_list
LabelsBoolDF['Notes'] = ""
LabelsBoolDF.loc[LabelsBoolDF[LabelsBoolDF['TRUE SUM']==25].index, 'Notes'] = "OK"

#rows in which not all TRUE (meaning this column/data does not exist for all year)
MissingData = LabelsBoolDF[LabelsBoolDF['TRUE SUM']!=25].index.tolist()

#Column names to keep
#LabelsBoolDF.loc[[0, 3:13, 15], 0].tolist()
Cols_to_keep_idx = [0]
Cols_to_keep_idx.extend(range(3,13))
Cols_to_keep_idx.extend((16, 18, 20, 25, 26))
Cols_to_keep_idx.extend(range(33, 49))
Cols_to_keep_idx.extend((64, 135, 137, 138, 183))
Cols_to_keep_idx.extend(range(185, 189))
Cols_to_keep_idx.extend(range(191, 199))
Cols_to_keep_idx.append(201)
Cols_to_keep_idx.extend(range(206, 219))
Cols_to_keep_idx.append(220)
Cols_to_keep_idx.extend(range(222, 229))
Cols_to_keep_idx.extend((230, 231, 237))
Cols_to_keep_idx.extend(range(237, 256))
Cols_to_keep_idx.extend(range(257, 260))
Cols_to_keep_idx.extend((262, 263))

Cols_to_keep_in_lib_dfs = LabelsBoolDF.iloc[Cols_to_keep_idx, 0].tolist()

#Make dictionaries of smaller dataframe with Cols_to_keep_in_lib_dfs ONLY
LibData_dict_reduced = dict()

for k, v in LibData_dict.items():
    LibData_dict_reduced[k] = pd.DataFrame()
    for i in Cols_to_keep_in_lib_dfs:
        if i in v:
            TempList = list(v[i])
            LibData_dict_reduced[k][i] = TempList

#Did above loop do what I thought it did?
print("Original dfs:")
for k, v in LibData_dict.items():
    print(k, v.shape)

print("\n", "Reduced dfs:")  
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

#Combine col names that mean same thing in different years into single 
Names_to_combine = ["ATTEND", "VISITS",  "C_LEGBAS", "C_LEGBASE", "DUPLI", "HRS_OPEN", "KIDATTEN", "KIDATTEND", 
                    "LIBRARIAN", "LIBRARIA", "POPU", "POPU_LSA", "POPU_UND", "POPU_UNDUP", "REFERENC", "REFERENCE", 
                    "STAFFEXP", "TOTEXP", "SUBSCRIP", "SUBSCRIPT", "TOTEXPCO", "TOTEXPCOL", 
                    "TOTOPEXP", "TOTOPEXP1", "TOTPEMP", "TOTSTAFF", "ZIP", "ZIP1"]

for i in range(0, len(Names_to_combine)-1, 2):
    for k, v in LibData_dict_reduced.items(): 
        if Names_to_combine[i] in v:
            name = Names_to_combine[i]
            newname = Names_to_combine[i] + "-" + Names_to_combine[i+1]
            v = v.rename(columns = {name: newname}, inplace = True)
        elif Names_to_combine[i+1] in v:
            name = Names_to_combine[i + 1]
            newname = Names_to_combine[i] + "-" + Names_to_combine[i+1]
            v = v.rename(columns = {name : newname}, inplace = True)

#Check that above code worked - names changes
for k, v in LibData_dict_reduced.items():
    print(v.columns)

for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

#Columns to add to DFs
"AUDIO is divided into AUDIO_PH, AUDIO_DL starting 2010. Source: 2010 Documentation."
"VIDEO was later split into VIDEO_DL and VIDEO_PH."

for k, v in LibData_dict_reduced.items(): 
    if "AUDIO_DL" in v:
        v["AUDIO"] = v['AUDIO_DL'] + v['AUDIO_PH']
    if "VIDEO_DL" in v:
        v["VIDEO"] = v['VIDEO_DL'] + v['VIDEO_PH']

#Check that # columns in some dataframes increased by 2 (expected)
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)
    
#Create DF w boolean values for prensence of each unique value in REDUCED dataframes
#THIS IS THE END GOAL OF THIS DATA

#Get all unique columns present in LibData_dict_reduced dataframes. 
ColumnsPresent_reduced = pd.DataFrame(index=range(0),columns=range(0))

for k, v in LibData_dict_reduced.items():
    dataframe_to_join_reduced = pd.DataFrame({k: list(v.columns)})
    ColumnsPresent_reduced = ColumnsPresent_reduced.join(dataframe_to_join_reduced, how = 'outer')

ColumnsPresent_reduced_str = ColumnsPresent_reduced.applymap(str)
ColumnsPresent_reduced_str = ColumnsPresent_reduced_str.reset_index(drop = True)
    
UniqueLabels_reduced = np.unique(ColumnsPresent_reduced_str.values)
LabelsBoolDF_reduced = pd.DataFrame(UniqueLabels_reduced)

#Create DF w boolean values for prensence of each unique value
#THIS IS THE END GOAL OF THIS DATA
for j in range(len(ColumnsPresent_reduced_str.columns)):
    lis = []
    for i in range(len(UniqueLabels_reduced)):
        lis.append(ColumnsPresent_reduced_str.iloc[:, j].isin([UniqueLabels_reduced[i]]).any())
    LabelsBoolDF_reduced[ColumnsPresent_reduced_str.columns[j]] = lis

## Sum all TRUE values. Value of 25 = all true
sum_list_reduced = []
for i in range(LabelsBoolDF_reduced.shape[0]):
    sum_list_reduced.append(sum(LabelsBoolDF_reduced.iloc[i, 1:26]))

#Label OK in Notes
LabelsBoolDF_reduced['TRUE SUM'] = sum_list_reduced
LabelsBoolDF_reduced['Notes'] = ""
LabelsBoolDF_reduced.loc[LabelsBoolDF_reduced[LabelsBoolDF_reduced['TRUE SUM']==25].index, 'Notes'] = "OK"

All_Cols_in_Reduced_dfs = list(LabelsBoolDF_reduced[0])
All_Cols_in_Reduced_dfs

for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)
    
#Delete columns deemed NOT useful for downstream analysis
NonUsefulColstoDelete = ['ENDDATE', 'LIBID', 'PHONE', 'POPU_UND-POPU_UNDUP', 'RSTATUS', 'TOTCOLL', 'ZIP-ZIP1']

LibData_dict_reduced_dropuseless = copy.deepcopy(LibData_dict_reduced)

for k, v in LibData_dict_reduced_dropuseless.items(): 
    for i in range(len(NonUsefulColstoDelete)):
        if NonUsefulColstoDelete[i] in v:
            v.drop([NonUsefulColstoDelete[i]], axis = 1, inplace = True)
            
#Check quickly whether columns were dropped
print("Reduced dfs:")
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

print("\n", "Reduced and dropped useless dfs:")
for k, v in LibData_dict_reduced_dropuseless.items():    
    print(k, v.shape)

#Get rid of negative values (replace w NaN)
#For all DFs, replace (-1, -3, -9) values with NaN. 
LibData_dict_reduced_dropuseless_NegRepWNaN = copy.deepcopy(LibData_dict_reduced_dropuseless)

for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    v[(v == -1) | (v == -3) | (v == -9)] = np.nan
    
#See if this worked (looks like it did)
print(sum(LibData_dict_reduced_dropuseless['_2014']['AUDIO_DL'] < 0))
print(np.where((LibData_dict_reduced_dropuseless['_2014']['AUDIO_DL'] < 0) == True))

print(sum(LibData_dict_reduced_dropuseless_NegRepWNaN['_2014']['AUDIO_DL'].isnull()))
print(np.where(LibData_dict_reduced_dropuseless_NegRepWNaN['_2014']['AUDIO_DL'].isnull()))

for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    print(k, v.shape)
    


_2014 (9305, 159) <class 'pandas.core.frame.DataFrame'>
_2015 (9251, 159) <class 'pandas.core.frame.DataFrame'>
_2016 (9252, 151) <class 'pandas.core.frame.DataFrame'>
_2001 (9133, 108) <class 'pandas.core.frame.DataFrame'>
_2002 (9141, 107) <class 'pandas.core.frame.DataFrame'>
_2003 (9214, 110) <class 'pandas.core.frame.DataFrame'>
_2004 (9210, 128) <class 'pandas.core.frame.DataFrame'>
_2005 (9201, 131) <class 'pandas.core.frame.DataFrame'>
_2006 (9211, 139) <class 'pandas.core.frame.DataFrame'>
_2007 (9217, 148) <class 'pandas.core.frame.DataFrame'>
_2008 (9284, 150) <class 'pandas.core.frame.DataFrame'>
_2009 (9299, 152) <class 'pandas.core.frame.DataFrame'>
_2010 (9308, 154) <class 'pandas.core.frame.DataFrame'>
_2011 (9315, 157) <class 'pandas.core.frame.DataFrame'>
_2012 (9305, 155) <class 'pandas.core.frame.DataFrame'>
_2013 (9309, 157) <class 'pandas.core.frame.DataFrame'>
_2000 (9078, 108) <class 'pandas.core.frame.DataFrame'>
_1992 (8944, 83) <class 'pandas.core.frame.DataF

_2014 (9305, 69)
_2015 (9251, 72)
_2016 (9252, 77)
_2001 (9133, 52)
_2002 (9141, 52)
_2003 (9214, 53)
_2004 (9210, 56)
_2005 (9201, 60)
_2006 (9211, 61)
_2007 (9217, 61)
_2008 (9284, 62)
_2009 (9299, 64)
_2010 (9308, 67)
_2011 (9315, 67)
_2012 (9305, 67)
_2013 (9309, 68)
_2000 (9078, 52)
_1992 (8944, 44)
_1993 (8929, 44)
_1994 (8920, 44)
_1995 (8981, 49)
_1996 (8946, 49)
_1997 (8968, 49)
_1998 (8966, 51)
_1999 (9048, 52)
Reduced dfs:
_2014 (9305, 69)
_2015 (9251, 72)
_2016 (9252, 77)
_2001 (9133, 52)
_2002 (9141, 52)
_2003 (9214, 53)
_2004 (9210, 56)
_2005 (9201, 60)
_2006 (9211, 61)
_2007 (9217, 61)
_2008 (9284, 62)
_2009 (9299, 64)
_2010 (9308, 67)
_2011 (9315, 67)
_2012 (9305, 67)
_2013 (9309, 68)
_2000 (9078, 52)
_1992 (8944, 44)
_1993 (8929, 44)
_1994 (8920, 44)
_1995 (8981, 49)
_1996 (8946, 49)
_1997 (8968, 49)
_1998 (8966, 51)
_1999 (9048, 52)

 Reduced and dropped useless dfs:
_2014 (9305, 63)
_2015 (9251, 66)
_2016 (9252, 70)
_2001 (9133, 47)
_2002 (9141, 47)
_2003 (9214, 48)


In [4]:
##CODE FROM NOTEBOOK: Library_Week2_Notebook4_Strategy#2
#(Load and process data created in Library_Week2_Notebook4_Strategy#2)

#Add usage column to each library
for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    v['Usage'] = v['ATTEND-VISITS']/v['POPU-POPU_LSA']
    
#Check that column was added
for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    print(k, v.shape)
    
#Remove all rows (libraries) for which Usage data missing for any window!
LibwUsage = copy.deepcopy(LibData_dict_reduced_dropuseless_NegRepWNaN)
for k, v in LibwUsage.items():
    NullInd = v[v.Usage.isnull()].index
    print(k, v.shape)
    v = v.drop(NullInd, axis = 0)
    print(k, v.shape)

_2014 (9305, 64)
_2015 (9251, 67)
_2016 (9252, 71)
_2001 (9133, 48)
_2002 (9141, 48)
_2003 (9214, 49)
_2004 (9210, 51)
_2005 (9201, 55)
_2006 (9211, 56)
_2007 (9217, 56)
_2008 (9284, 57)
_2009 (9299, 59)
_2010 (9308, 62)
_2011 (9315, 62)
_2012 (9305, 62)
_2013 (9309, 63)
_2000 (9078, 48)
_1992 (8944, 40)
_1993 (8929, 40)
_1994 (8920, 40)
_1995 (8981, 45)
_1996 (8946, 45)
_1997 (8968, 45)
_1998 (8966, 47)
_1999 (9048, 48)
_2014 (9305, 64)
_2014 (9259, 64)
_2015 (9251, 67)
_2015 (9231, 67)
_2016 (9252, 71)
_2016 (9234, 71)
_2001 (9133, 48)
_2001 (9131, 48)
_2002 (9141, 48)
_2002 (9138, 48)
_2003 (9214, 49)
_2003 (9214, 49)
_2004 (9210, 51)
_2004 (9207, 51)
_2005 (9201, 55)
_2005 (9198, 55)
_2006 (9211, 56)
_2006 (9208, 56)
_2007 (9217, 56)
_2007 (9214, 56)
_2008 (9284, 57)
_2008 (9257, 57)
_2009 (9299, 59)
_2009 (9256, 59)
_2010 (9308, 62)
_2010 (9273, 62)
_2011 (9315, 62)
_2011 (9275, 62)
_2012 (9305, 62)
_2012 (9265, 62)
_2013 (9309, 63)
_2013 (9263, 63)
_2000 (9078, 48)
_2000 (9078, 4

In [8]:
##CODE FROM NOTEBOOK: Library_Week2_Notebook4_Strategy#2
#(Load and process data created in Library_Week2_Notebook4_Strategy#2)

#Create DF w boolean values for prensence of each unique value in REDUCED dataframes
#THIS IS THE END GOAL OF THIS DATA

#Get all unique columns present in LibData_dict_reduced dataframes. 
ColumnsPresent_reduced = pd.DataFrame(index=range(0),columns=range(0))

for k, v in LibwUsage .items():
    dataframe_to_join_reduced = pd.DataFrame({k: list(v.columns)})
    ColumnsPresent_reduced = ColumnsPresent_reduced.join(dataframe_to_join_reduced, how = 'outer')
    
UniqueLabels_reduced = np.unique(ColumnsPresent_reduced_str.values)
LabelsBoolDF_reduced = pd.DataFrame(UniqueLabels_reduced)

#Create DF w boolean values for prensence of each unique value
#THIS IS THE END GOAL OF THIS DATA
for j in range(len(ColumnsPresent_reduced_str.columns)):
    lis = []
    for i in range(len(UniqueLabels_reduced)):
        lis.append(ColumnsPresent_reduced_str.iloc[:, j].isin([UniqueLabels_reduced[i]]).any())
    LabelsBoolDF_reduced[ColumnsPresent_reduced_str.columns[j]] = lis
    
#Reorder columns
colnames = LabelsBoolDF_reduced.columns.tolist()
col_order = colnames[0:1] + colnames[18:26] + colnames[17:18] + colnames[4:17] + colnames[1:4]
LabelsBoolDF_reduced = LabelsBoolDF_reduced[col_order]

## Sum all TRUE values. Value of 25 = all true
sum_list_reduced_all = []
for i in range(LabelsBoolDF_reduced.shape[0]):
    sum_list_reduced_all.append(sum(LabelsBoolDF_reduced.iloc[i, 1:26]))
    
sum_list_reduced_ten = []
for i in range(LabelsBoolDF_reduced.shape[0]):
    sum_list_reduced_ten.append(sum(LabelsBoolDF_reduced.iloc[i, 15:26]))
    
sum_list_reduced_six = []
for i in range(LabelsBoolDF_reduced.shape[0]):
    sum_list_reduced_six.append(sum(LabelsBoolDF_reduced.iloc[i, 19:26]))

#Label OK in Notes
LabelsBoolDF_reduced['All_Years_Present'] = sum_list_reduced_all
LabelsBoolDF_reduced['2006_2016_Present'] = sum_list_reduced_ten
LabelsBoolDF_reduced['2010_2016_Present'] = sum_list_reduced_six

#Based on this, I will use 2010-2016 DF. This has the most values.
LabelsBoolDF_reduced.loc[29: ,:]

#Rename first column
LabelsBoolDF_reduced.rename(columns = {LabelsBoolDF_reduced.columns[0] : "Col"}, inplace = True)

In [9]:
##CODE FROM NOTEBOOK: Library_Week2_Notebook4_Strategy#2
#(Load and process data created in Library_Week2_Notebook4_Strategy#2)

#Make sure each df has all 83 columns. If column did not exist, add this column with NaN
Unique_col_names_list = list(LabelsBoolDF_reduced['Col'])

LibwUsage_2 = copy.deepcopy(LibwUsage)

for k, v in LibwUsage_2.items():
    for i in range(len(Unique_col_names_list)):
        if Unique_col_names_list[i] not in v:
            v[Unique_col_names_list[i]] = np.nan

for k, v in LibwUsage_2.items():
    print(k, v.shape)

_2014 (9305, 85)
_2015 (9251, 85)
_2016 (9252, 85)
_2001 (9133, 85)
_2002 (9141, 85)
_2003 (9214, 85)
_2004 (9210, 85)
_2005 (9201, 85)
_2006 (9211, 85)
_2007 (9217, 85)
_2008 (9284, 85)
_2009 (9299, 85)
_2010 (9308, 85)
_2011 (9315, 85)
_2012 (9305, 85)
_2013 (9309, 85)
_2000 (9078, 85)
_1992 (8944, 85)
_1993 (8929, 85)
_1994 (8920, 85)
_1995 (8981, 85)
_1996 (8946, 85)
_1997 (8968, 85)
_1998 (8966, 85)
_1999 (9048, 85)


In [12]:
for k, v in LibwUsage.items():
    print(k, v.shape)

_2014 (9305, 64)
_2015 (9251, 67)
_2016 (9252, 71)
_2001 (9133, 48)
_2002 (9141, 48)
_2003 (9214, 49)
_2004 (9210, 51)
_2005 (9201, 55)
_2006 (9211, 56)
_2007 (9217, 56)
_2008 (9284, 57)
_2009 (9299, 59)
_2010 (9308, 62)
_2011 (9315, 62)
_2012 (9305, 62)
_2013 (9309, 63)
_2000 (9078, 48)
_1992 (8944, 40)
_1993 (8929, 40)
_1994 (8920, 40)
_1995 (8981, 45)
_1996 (8946, 45)
_1997 (8968, 45)
_1998 (8966, 47)
_1999 (9048, 48)


## Here is where I start to put last year's usage in 

In [49]:
#Include previous usage in dataframes
dftemp = pd.DataFrame()
LibwUsage_addlastyearusage = dict()
LibwUsage_copy = copy.deepcopy(LibwUsage_2)

year_list = [str('_2009'), str('_2010'), str('_2011'), str('_2012'), str('_2013'), str('_2014'), str('_2015'), str('_2016')]
for i in range(len(year_list)-1):
    print(year_list[i])
    dftemp = LibwUsage_copy[year_list[i]][['FSCSKEY', 'Usage']]
    dftemp.rename(columns = {"Usage": "Usage_PrevYear"}, inplace = True)
    LibwUsage_addlastyearusage[year_list[i+1]] = pd.merge(LibwUsage_copy[year_list[i+1]], dftemp, how = "outer", on = 'FSCSKEY')
    

_2009
_2010
_2011


//anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


_2012
_2013
_2014
_2015


In [55]:
#Concatenate 2010-2016
Concat201116 = pd.concat([LibwUsage_addlastyearusage['_2010'], LibwUsage_addlastyearusage['_2011'], LibwUsage_addlastyearusage['_2012'], LibwUsage_addlastyearusage['_2013'], LibwUsage_addlastyearusage['_2014'], LibwUsage_addlastyearusage['_2015'], LibwUsage_addlastyearusage['_2016']], axis=0, sort = False, join = "outer")
print(Concat201116.shape)

#rename first col
LabelsBoolDF_reduced.rename(columns = {LabelsBoolDF_reduced.columns[0] : "Col"}, inplace = True)

#Which features are not present in all 7 years?
Index = LabelsBoolDF_reduced[LabelsBoolDF_reduced['2010_2016_Present'] != 7].index

#Delete these features.
Cols_to_delete = LabelsBoolDF_reduced.loc[Index, 'Col']

#Delete from dataframe:
Cols_to_del_list = list(Cols_to_delete[:-1])
print("Columns to delete:", Cols_to_del_list)

#Delete columns
Concat201116_INCOMPLETECOLSDEL = Concat201116.drop(columns = Cols_to_del_list)

print(Concat201116.shape)
print(Concat201116_INCOMPLETECOLSDEL.shape)

#Nonuseful columns to delete from above:
NonUsefulColstoDelete = ['ENDDATE', 'LIBID', 'PHONE', 'POPU_UND-POPU_UNDUP', 'RSTATUS', 'ZIP-ZIP1', 'nan']
Concat201116_INCOMPLETECOLSDEL2 = Concat201116_INCOMPLETECOLSDEL.drop(columns = NonUsefulColstoDelete)
print(Concat201116_INCOMPLETECOLSDEL2.shape)

Concat201116_INCOMPLETECOLSDEL2.columns

#I SHOULD CHECK THIS MORE CAREFULLY ON FRI!!

#Columns to move to front, not use 
'ADDRESS', 'ATTEND-VISITS', 'CITY', 'CNTY', 'FSCSKEY', 'LIBNAME',   

#Columns to delete prior to analysis
'AUDIO_DL', 'AUDIO_PH'

#Columns to categorize -- DELETE FOR NOW, but for tomorrow may was to categorize
'GEOCODE', 'LOCALE', 'OBEREG', 'STABR', 'C_LEGBAS-C_LEGBASE'

#Delete 'AUDIO_DL', 'AUDIO_PH'
Concat201116_INCOMPLETECOLSDEL3 = Concat201116_INCOMPLETECOLSDEL2.drop(columns = ['AUDIO_DL', 'AUDIO_PH'])
#Put 'ADDRESS', 'ATTEND-VISITS', 'CITY', 'CNTY', 'FSCSKEY', 'LIBNAME' in dif DF
Location_Detail_DF = Concat201116_INCOMPLETECOLSDEL2[['ADDRESS', 'ATTEND-VISITS', 'CITY', 'CNTY', 'FSCSKEY', 'LIBNAME']]

#NOW delete from concatDF
Concat201116_INCOMPLETECOLSDEL4 = Concat201116_INCOMPLETECOLSDEL3.drop(columns = ['ADDRESS', 'ATTEND-VISITS', 'CITY', 'CNTY', 'FSCSKEY', 'LIBNAME'])

#NOW concat 4 values
Concat_w_dummies = pd.get_dummies(Concat201116_INCOMPLETECOLSDEL4, columns=['GEOCODE', 'LOCALE', 'OBEREG', 'STABR', 'C_LEGBAS-C_LEGBASE'], prefix=['GEOCODE', 'LOCALE', 'OBEREG', 'STABR', 'C_LEGBAS-C_LEGBASE'])

print(Concat201116_INCOMPLETECOLSDEL2.shape, Concat201116_INCOMPLETECOLSDEL3.shape, Concat201116_INCOMPLETECOLSDEL4.shape, Concat_w_dummies.shape)

(65271, 86)
Columns to delete: ['EC_LO_OT', 'EC_ST', 'ELACCEXP', 'ELCONT', 'ELECCOLL', 'ELINFO', 'ELMATCIR', 'ELMATS', 'ELSVCACC', 'ERES_USR', 'ESUBSCRP', 'INETACC', 'PHYSCIR', 'REAPLOCALE', 'TOTCOLL', 'WIFISESS']
(65271, 86)
(65271, 70)
(65271, 63)
(65271, 63) (65271, 61) (65271, 55) (65271, 161)


In [56]:
###RESET INDEX for BOTH
Concat_w_dummies = Concat_w_dummies.reset_index(drop = True)
Location_Detail_DF = Location_Detail_DF.reset_index(drop = True)

#25000 rows have NaN. Just DELETE These rows for now
print(Concat_w_dummies.shape)
print(len(pd.isnull(Concat_w_dummies).any(1).nonzero()[0]))

idx = pd.isnull(Concat_w_dummies).any(1).nonzero()[0]

#Delete_row_w_any_NaN
Concat_w_dummies_noNaN = Concat_w_dummies.drop(idx)
Location_Detail_DF_noNaN = Location_Detail_DF.drop(idx)

print(Concat_w_dummies_noNaN.shape, Location_Detail_DF_noNaN.shape)

(65271, 161)
25350
(39921, 161) (39921, 6)


In [57]:
##CODE FROM NOTEBOOK: Library_Week2_Notebook4_Strategy#2
#(Load and process data created in Library_Week2_Notebook4_Strategy#2)

#Above, I found that 4 columns - 'OTHOPEXP', 'SALARIES', 'STAFFEXP-TOTEXP', 'BENEFIT' - are objects, meaning some balues are non-numeric
#(Not shown explicitly in this notebook, but shown in previous notebook)
#It looks like some are left blank.
print(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.OTHOPEXP == ' '].index)
print(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.SALARIES == ' '].index)
print(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN['STAFFEXP-TOTEXP'] == ' '].index)
print(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.BENEFIT == ' '].index)

#Are all indices the same? YES. 
print(len(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.OTHOPEXP == ' '].index.intersection(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.SALARIES == ' '].index)))
print(len(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.OTHOPEXP == ' '].index.intersection(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN['STAFFEXP-TOTEXP'] == ' '].index)))
print(len(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.OTHOPEXP == ' '].index.intersection(Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.BENEFIT == ' '].index)))

IndexToDropSpaces = Concat_w_dummies_noNaN[Concat_w_dummies_noNaN.OTHOPEXP == ' '].index

Concat_w_dummies_noNaN_noSpace = Concat_w_dummies_noNaN.drop(IndexToDropSpaces)
Location_Detail_DF_noNaN_noSpace = Location_Detail_DF_noNaN.drop(IndexToDropSpaces)
print(Concat_w_dummies_noNaN.shape, Location_Detail_DF_noNaN.shape)
print(Concat_w_dummies_noNaN_noSpace.shape, Location_Detail_DF_noNaN_noSpace.shape)

Int64Index([37344, 37346, 37349, 37350, 37351, 37352, 37354, 37357, 37358,
            37360,
            ...
            46594, 46595, 46597, 46606, 46608, 46609, 46610, 46617, 46623,
            46624],
           dtype='int64', length=3295)
Int64Index([37344, 37346, 37349, 37350, 37351, 37352, 37354, 37357, 37358,
            37360,
            ...
            46594, 46595, 46597, 46606, 46608, 46609, 46610, 46617, 46623,
            46624],
           dtype='int64', length=3295)
Int64Index([37344, 37346, 37349, 37350, 37351, 37352, 37354, 37357, 37358,
            37360,
            ...
            46594, 46595, 46597, 46606, 46608, 46609, 46610, 46617, 46623,
            46624],
           dtype='int64', length=3295)
Int64Index([37344, 37346, 37349, 37350, 37351, 37352, 37354, 37357, 37358,
            37360,
            ...
            46594, 46595, 46597, 46606, 46608, 46609, 46610, 46617, 46623,
            46624],
           dtype='int64', length=3295)
3295
3295
3295
(39921, 1

## Run first RF. No outliers remove. 

In [78]:
#Remove the one very obvious outlier
Idx = Concat_w_dummies_noNaN_noSpace[Concat_w_dummies_noNaN_noSpace.Usage > 1600].index

Location_Detail_DF_noNaN_noSpace_1611dropped = Location_Detail_DF_noNaN_noSpace.drop(Idx)
Concat_w_dummies_noNaN_noSpace_1611dropped = Concat_w_dummies_noNaN_noSpace.drop(Idx)

Location_Detail_DF_noNaN_noSpace_1611dropped = Location_Detail_DF_noNaN_noSpace_1611dropped.reset_index(drop = True)
Concat_w_dummies_noNaN_noSpace_1611dropped = Concat_w_dummies_noNaN_noSpace_1611dropped.reset_index(drop = True)


In [79]:
#Take out 20% of libraries randomly for testing

import random
print(Concat_w_dummies_noNaN.shape)
print(Concat_w_dummies_noNaN_noSpace_1611dropped.shape)
print(Location_Detail_DF_noNaN.shape)
print(Location_Detail_DF_noNaN_noSpace_1611dropped.shape)

Concat_w_dummies_noNaN_noSpace_1611dropped_ri = Concat_w_dummies_noNaN_noSpace_1611dropped.reset_index(drop = True)
Location_Detail_DF_noNaN_noSpace_1611dropped = Location_Detail_DF_noNaN_noSpace_1611dropped.reset_index(drop = True)

UniqueFSCS = set(Location_Detail_DF_noNaN_noSpace_1611dropped.loc[:, 'FSCSKEY'].tolist())
print(len(UniqueFSCS))

#Pick 20% of FSCSKEYS randomly
Numb_to_sel = int((.20*len(UniqueFSCS)))
_20PctFSCSList = random.sample(UniqueFSCS, Numb_to_sel)

#Get index of these FSCSKEYS from Location_Detail_DF_noNaN_noSpace_ri
Indexlist = list()
for FSCSkey in _20PctFSCSList:
    Indexfor_20PctFSCSList = Location_Detail_DF_noNaN_noSpace_1611dropped[Location_Detail_DF_noNaN_noSpace_1611dropped.FSCSKEY == FSCSkey].index
    Indexlist.append(list(Indexfor_20PctFSCSList))

flattened_Indexlist = [val for sublist in Indexlist for val in sublist]
print(len(flattened_Indexlist))

(39921, 161)
(36625, 161)
(39921, 6)
(36625, 6)
6293
7306


In [ ]:
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 10, max_depth = 60)

X = Concat_w_dummies_noNaN_noSpace_1611dropped_ri.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_1611dropped_ri['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#Train model on TRAIN set
rfmodel.fit(X_train, y_train);

#Test model on TEST set (this is the right this to do)
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)
#df of predictions, labels and errors
df_TEST = pd.DataFrame()
df_TEST['Usage_val'] = y_test
df_TEST['Predictions'] = predictions
df_TEST['errors'] = (list(errors))

#Test model on TRAIN set (this is technically the wrong thing to do, but still informative)
predictions = rfmodel.predict(X_train)
errors = abs(predictions - y_train)
#df of predictions, labels and errors
df_TRAIN = pd.DataFrame()
df_TRAIN['Usage_val'] = y_train
df_TRAIN['Predictions'] = predictions
df_TRAIN['errors'] = (list(errors))

#Get feature importance
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#Plot df_TEST Usage vs. Prediction:
# Data
x = df_TEST.iloc[:, 0]
y = df_TEST.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

#Plot df_TRAIN Usage vs. Prediction:
# Data
x = df_TRAIN.iloc[:, 0]
y = df_TRAIN.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))

In [ ]:
#Below is old library

## Random Forest Model #1 - code condensed from Library_Week2_Notebook4_Strategy#2

In [ ]:
#Take out 20% of libraries randomly for testing

import random
print(Concat_w_dummies_noNaN.shape)
print(Concat_w_dummies_noNaN_noSpace.shape)
print(Location_Detail_DF_noNaN.shape)
print(Location_Detail_DF_noNaN_noSpace.shape)

Concat_w_dummies_noNaN_noSpace_ri = Concat_w_dummies_noNaN_noSpace.reset_index(drop = True)
Location_Detail_DF_noNaN_noSpace_ri = Location_Detail_DF_noNaN_noSpace.reset_index(drop = True)

UniqueFSCS = set(Location_Detail_DF_noNaN_noSpace_ri.loc[:, 'FSCSKEY'].tolist())
print(len(UniqueFSCS))

#Pick 20% of FSCSKEYS randomly
Numb_to_sel = int((.20*len(UniqueFSCS)))
_20PctFSCSList = random.sample(UniqueFSCS, Numb_to_sel)

#Get index of these FSCSKEYS from Location_Detail_DF_noNaN_noSpace_ri
Indexlist = list()
for FSCSkey in _20PctFSCSList:
    Indexfor_20PctFSCSList = Location_Detail_DF_noNaN_noSpace_ri[Location_Detail_DF_noNaN_noSpace_ri.FSCSKEY == FSCSkey].index
    Indexlist.append(list(Indexfor_20PctFSCSList))

flattened_Indexlist = [val for sublist in Indexlist for val in sublist]
print(len(flattened_Indexlist))

In [ ]:
#RF with SPECIFIC libraries removed.
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42)

X = Concat_w_dummies_noNaN_noSpace_ri.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rfmodel.fit(X_train, y_train);

predictions = rfmodel.predict(X_test)

errors = abs(predictions - y_test)

#df of predictions, labels and errors
df = pd.DataFrame(predictions, y_test)
df['errors'] = (list(errors))

In [ ]:
print(rfmodel.score(X_test, y_test))
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

In [ ]:
from numpy.polynomial.polynomial import polyfit

# Sample data
x = df.iloc[:, 0]
y = df.iloc[:, 1]

# Fit with polyfit
b, m = polyfit(x, y, 1)

plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.xlim((0, 30))
plt.ylim((0, 40))
plt.show()


In [ ]:
#Can out-of-bag estimates substitute for a separate test set??
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42, oob_score=True)

X = Concat_w_dummies_noNaN_noSpace_ri.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri['Usage']

rfmodel.fit(X, y);

oob_error = 1 - rfmodel.oob_score_

predictions = rfmodel.predict(X)
errors = abs(predictions - y)

#df of predictions, labels and errors
df = pd.DataFrame()
df['Usage_val'] = y
df['Predictions'] = predictions
df['errors'] = (list(errors))

#Probably not...oob error is VERY large. So training and testing on same dataset is not valid. (Although if I train and test on same set, then rsults are VERY good and capital is best predictor. Does this tell me anything??)

In [ ]:
print(rfmodel.oob_score_)

feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)


In [ ]:
df

In [ ]:
from numpy.polynomial.polynomial import polyfit

# Data
x = df.iloc[:, 0]
y = df.iloc[:, 1]

# Fit with polyfit
b, m = polyfit(x, y, 1)

plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.xlim(0,200)
plt.ylim(0,200)
plt.show()

In [ ]:
#Go back to RF with SPECIFIC libraries removed.
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42)

X = Concat_w_dummies_noNaN_noSpace_ri.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rfmodel.fit(X_train, y_train);

predictions = rfmodel.predict(X_test)

errors = abs(predictions - y_test)

#df of predictions, labels and errors
df = pd.DataFrame()
df['Usage_val'] = y_test
df['Predictions'] = predictions
df['errors'] = (list(errors))

# Data
x = df.iloc[:, 0]
y = df.iloc[:, 1]

# Fit with polyfit
b, m = polyfit(x, y, 1)

plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

In [ ]:
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.ylim(0, 40)
plt.xlim(0, 40)
plt.show()

In [ ]:
df['Usage_bin'] = pd.qcut(df['Usage_val'], 4)
df['Predictions_bin'] = pd.qcut(df['Predictions'], 4)

df = df.reset_index(drop = True)
df['Usage_bin_val'] = ""
df['Predictions_bin_val'] = ""

Usage_bin_list = df['Usage_bin'].unique()
Predictions_bin_list = df['Predictions_bin'].unique()

for j in range(df.shape[0]):
    for i in range(4):
        if str(df.loc[j, 'Usage_bin']) == str(Usage_bin_list[i]):
            df.loc[j, 'Usage_bin_val'] = i+1 
        if str(df.loc[j, 'Predictions_bin']) == str(Predictions_bin_list[i]):
            df.loc[j, 'Predictions_bin_val'] = i+1 
                               

In [ ]:
sum_equal = 0

for i in range(df.shape[0]):
    if df.loc[i, 'Usage_bin_val'] == df.loc[i, 'Predictions_bin_val']:
        sum_equal = sum_equal+1
print(sum_equal, df.shape[0])

In [ ]:
df["Difference"] = abs(df['Usage_bin_val'] - df['Predictions_bin_val'])

In [ ]:
df.hist(column='Difference')

In [ ]:
df['error_pct'] = df['errors']/df['Usage_val']

sum_error = 0

for i in range(df.shape[0]):
    if df.loc[i, 'error_pct'] <= .3:
        sum_error = sum_error + 1
print(sum_error, df.shape[0])

In [ ]:
df

In [ ]:
print(feature_importances)

In [ ]:
df.hist(column='Usage_val', bins = len(df['Usage_val'].unique()))

In [ ]:
#RF with SPECIFIC libraries removed, but POPU-POPU_LSA kept in.
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42)

X = Concat_w_dummies_noNaN_noSpace_ri.drop(columns = ['Usage'])
y = Concat_w_dummies_noNaN_noSpace_ri['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rfmodel.fit(X_train, y_train);

predictions = rfmodel.predict(X_test)

errors = abs(predictions - y_test)

#df of predictions, labels and errors
df = pd.DataFrame()
df['Usage_val'] = y_test
df['Predictions'] = predictions
df['errors'] = (list(errors))

# Data
x = df.iloc[:, 0]
y = df.iloc[:, 1]

# Fit with polyfit
b, m = polyfit(x, y, 1)

plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#If I put back population, it looks like prediction improves a lot. BUT that's super unfair and I won't actually get anything meaningful out of it. 

In [ ]:
#Next, try RF with train and test set (without removing specific libraries)

#RF with SPECIFIC libraries removed, but POPU-POPU_LSA kept in.
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42)

X = Concat_w_dummies_noNaN_noSpace_ri.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri['Usage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

rfmodel.fit(X_train, y_train);
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)

#df of predictions, labels and errors
df = pd.DataFrame()
df['Usage_val'] = y_test
df['Predictions'] = predictions
df['errors'] = (list(errors))

# Data
x = df.iloc[:, 0]
y = df.iloc[:, 1]

# Fit with polyfit
b, m = polyfit(x, y, 1)

plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

In [ ]:
df

In [ ]:
df = df.reset_index(drop = True)
df['error_pct'] = df['errors']/df['Usage_val']

sum_error = 0

for i in range(df.shape[0]):
    if df.loc[i, 'error_pct'] <= .35:
        sum_error = sum_error + 1
#print(sum_error, df.shape[0])

In [ ]:
#Get rid of usage 0 to get mean error
df[df['Usage_val'] == 0].index

In [ ]:
df = df.drop(df[df['Usage_val'] == 0].index)

In [ ]:
df['error_pct'].mean()

In [ ]:
list(df['error_pct'])

In [ ]:
df['error_pct'].max()

In [ ]:
df[df['error_pct'] > 8000].index

In [ ]:
df.loc[3502, :]

## After 2:30 standup

### I should run the model the right way. Take 20% of libraries out first!

In [ ]:
###GET random set of libraries to remove
UniqueFSCS = set(Location_Detail_DF_noNaN_noSpace_ri.loc[:, 'FSCSKEY'].tolist())
print(len(UniqueFSCS))

#Pick 20% of FSCSKEYS randomly
Numb_to_sel = int((.20*len(UniqueFSCS)))
_20PctFSCSList = random.sample(UniqueFSCS, Numb_to_sel)

#Get index of these FSCSKEYS from Location_Detail_DF_noNaN_noSpace_ri
Indexlist = list()
for FSCSkey in _20PctFSCSList:
    Indexfor_20PctFSCSList = Location_Detail_DF_noNaN_noSpace_ri[Location_Detail_DF_noNaN_noSpace_ri.FSCSKEY == FSCSkey].index
    Indexlist.append(list(Indexfor_20PctFSCSList))
flattened_Indexlist = [val for sublist in Indexlist for val in sublist]
print(len(flattened_Indexlist))

In [ ]:
#RF with SPECIFIC libraries removed, but POPU-POPU_LSA kept in.
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42)

X = Concat_w_dummies_noNaN_noSpace_ri.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#Train model on TRAIN set
rfmodel.fit(X_train, y_train);

#Test model on TEST set (this is the right this to do)
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)
#df of predictions, labels and errors
df_TEST = pd.DataFrame()
df_TEST['Usage_val'] = y_test
df_TEST['Predictions'] = predictions
df_TEST['errors'] = (list(errors))

#Test model on TRAIN set (this is technically the wrong thing to do, but still informative)
predictions = rfmodel.predict(X_train)
errors = abs(predictions - y_train)
#df of predictions, labels and errors
df_TRAIN = pd.DataFrame()
df_TRAIN['Usage_val'] = y_train
df_TRAIN['Predictions'] = predictions
df_TRAIN['errors'] = (list(errors))

#Get feature importance
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#Plot df_TEST Usage vs. Prediction:
# Data
x = df.iloc[:, 0]
y = df.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

In [ ]:
print(rfmodel.score(X_test, y_test))
print(rfmodel.score(X_train, y_train))

In [ ]:
print(df_TEST['errors'].max())
print(df_TRAIN['errors'].max())
idx = df_TEST[df_TEST['errors'] > 1600].index
print(idx)
df_TEST.loc[33595, :]

In [ ]:
y_test.max()

### Take out usage outliers! (As shown above, there are definitely some!)

In [ ]:
print(Location_Detail_DF_noNaN_noSpace_ri.shape)
print(Concat_w_dummies_noNaN_noSpace_ri.shape)

In [ ]:
#Remove anything 5SD's higher than mean?
mean_plus_5xSD = Concat_w_dummies_noNaN_noSpace_ri.Usage.mean() + 5*Concat_w_dummies_noNaN_noSpace_ri.Usage.std()
OutlierIdx = Concat_w_dummies_noNaN_noSpace_ri[Concat_w_dummies_noNaN_noSpace_ri.Usage > mean_plus_5xSD].index
Concat_w_dummies_noNaN_noSpace_ri.loc[OutlierIdx, ['Usage']]



In [ ]:
#Actually, for starters let's simply remove row 33595, with usage >1600
Location_Detail_DF_noNaN_noSpace_ri_1611dropped = Location_Detail_DF_noNaN_noSpace_ri.drop([33595])
Concat_w_dummies_noNaN_noSpace_ri_1611dropped = Concat_w_dummies_noNaN_noSpace_ri.drop([33595])

Location_Detail_DF_noNaN_noSpace_ri_1611dropped = Location_Detail_DF_noNaN_noSpace_ri_1611dropped.reset_index(drop = True)
Concat_w_dummies_noNaN_noSpace_ri_1611dropped = Concat_w_dummies_noNaN_noSpace_ri_1611dropped.reset_index(drop = True)



In [ ]:
###GET random set of libraries to remove
UniqueFSCS = set(Location_Detail_DF_noNaN_noSpace_ri_1611dropped.loc[:, 'FSCSKEY'].tolist())
print(len(UniqueFSCS))

#Pick 20% of FSCSKEYS randomly
Numb_to_sel = int((.20*len(UniqueFSCS)))
_20PctFSCSList = random.sample(UniqueFSCS, Numb_to_sel)

#Get index of these FSCSKEYS from Location_Detail_DF_noNaN_noSpace_ri
Indexlist = list()
for FSCSkey in _20PctFSCSList:
    Indexfor_20PctFSCSList = Location_Detail_DF_noNaN_noSpace_ri_1611dropped[Location_Detail_DF_noNaN_noSpace_ri_1611dropped.FSCSKEY == FSCSkey].index
    Indexlist.append(list(Indexfor_20PctFSCSList))
flattened_Indexlist = [val for sublist in Indexlist for val in sublist]
print(len(flattened_Indexlist))

In [ ]:
#RF with SPECIFIC libraries removed (and POPU-POPU_LSA removed too). [Single outlier dropped]
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42)

X = Concat_w_dummies_noNaN_noSpace_ri_1611dropped.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_1611dropped['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#Train model on TRAIN set
rfmodel.fit(X_train, y_train);

#Test model on TEST set (this is the right this to do)
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)
#df of predictions, labels and errors
df_TEST = pd.DataFrame()
df_TEST['Usage_val'] = y_test
df_TEST['Predictions'] = predictions
df_TEST['errors'] = (list(errors))

#Test model on TRAIN set (this is technically the wrong thing to do, but still informative)
predictions = rfmodel.predict(X_train)
errors = abs(predictions - y_train)
#df of predictions, labels and errors
df_TRAIN = pd.DataFrame()
df_TRAIN['Usage_val'] = y_train
df_TRAIN['Predictions'] = predictions
df_TRAIN['errors'] = (list(errors))

#Get feature importance
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#Plot df_TEST Usage vs. Prediction:
# Data
x = df_TEST.iloc[:, 0]
y = df_TEST.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))

In [ ]:
print(df_TEST['errors'].max())
print(df_TRAIN['errors'].max())

In [ ]:
#take out outliers!!
#plot correlation map
#email Jolene

In [ ]:
df_TEST.loc[df_TEST[df_TEST['errors'] > 90].index, :]

In [ ]:
FSCSKeys_Used_Above_Jan28_535pm = _20PctFSCSList

In [ ]:
#Now, try to remove anything 5SD's higher than mean.
Concat_w_dummies_noNaN_noSpace_ri = Concat_w_dummies_noNaN_noSpace_ri.reset_index(drop = True)
Location_Detail_DF_noNaN_noSpace_ri = Location_Detail_DF_noNaN_noSpace_ri.reset_index(drop = True)

mean_plus_5xSD = Concat_w_dummies_noNaN_noSpace_ri.Usage.mean() + 5*Concat_w_dummies_noNaN_noSpace_ri.Usage.std()
OutlierIdx = Concat_w_dummies_noNaN_noSpace_ri[Concat_w_dummies_noNaN_noSpace_ri.Usage > mean_plus_5xSD].index
print("Number of outliers:", Concat_w_dummies_noNaN_noSpace_ri.loc[OutlierIdx, ['Usage']].shape[0])
      
Concat_w_dummies_noNaN_noSpace_ri_DropOutliers=Concat_w_dummies_noNaN_noSpace_ri.drop(OutlierIdx)
Location_Detail_DF_noNaN_noSpace_ri_DropOutliers=Location_Detail_DF_noNaN_noSpace_ri.drop(OutlierIdx)

print(Concat_w_dummies_noNaN_noSpace_ri.shape, Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.shape)
print(Location_Detail_DF_noNaN_noSpace_ri.shape, Location_Detail_DF_noNaN_noSpace_ri_DropOutliers.shape)

In [ ]:
###GET random set of libraries to remove
Location_Detail_DF_noNaN_noSpace_ri_DropOutliers = Location_Detail_DF_noNaN_noSpace_ri_DropOutliers.reset_index(drop=True)
Concat_w_dummies_noNaN_noSpace_ri_DropOutliers = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.reset_index(drop=True)

UniqueFSCS = set(Location_Detail_DF_noNaN_noSpace_ri_DropOutliers.loc[:, 'FSCSKEY'].tolist())
print(len(UniqueFSCS))

#Pick 20% of FSCSKEYS randomly
Numb_to_sel = int((.20*len(UniqueFSCS)))
_20PctFSCSList = random.sample(UniqueFSCS, Numb_to_sel)

#Get index of these FSCSKEYS from Location_Detail_DF_noNaN_noSpace_ri
Indexlist = list()
for FSCSkey in _20PctFSCSList:
    Indexfor_20PctFSCSList = Location_Detail_DF_noNaN_noSpace_ri_DropOutliers[Location_Detail_DF_noNaN_noSpace_ri_DropOutliers.FSCSKEY == FSCSkey].index
    Indexlist.append(list(Indexfor_20PctFSCSList))
flattened_Indexlist = [val for sublist in Indexlist for val in sublist]
print(len(flattened_Indexlist))

In [ ]:
#RF with SPECIFIC libraries removed (and POPU-POPU_LSA removed too).
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42)

X = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#Train model on TRAIN set
rfmodel.fit(X_train, y_train);

#Test model on TEST set (this is the right this to do)
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)
#df of predictions, labels and errors
df_TEST = pd.DataFrame()
df_TEST['Usage_val'] = y_test
df_TEST['Predictions'] = predictions
df_TEST['errors'] = (list(errors))

#Test model on TRAIN set (this is technically the wrong thing to do, but still informative)
predictions = rfmodel.predict(X_train)
errors = abs(predictions - y_train)
#df of predictions, labels and errors
df_TRAIN = pd.DataFrame()
df_TRAIN['Usage_val'] = y_train
df_TRAIN['Predictions'] = predictions
df_TRAIN['errors'] = (list(errors))

#Get feature importance
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#Plot df_TEST Usage vs. Prediction:
# Data
x = df_TEST.iloc[:, 0]
y = df_TEST.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))

In [ ]:
#Plot df_TEST Usage vs. Prediction:
# Data
x = df_TRAIN.iloc[:, 0]
y = df_TRAIN.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

In [ ]:
FSCSKeys_Used_Above_Jan28_550pm = _20PctFSCSList

In [ ]:
df_TEST['error_pct'] = df_TEST['errors']/df_TEST['Usage_val']

sum_error = 0

df_TEST = df_TEST.reset_index(drop = True)

for i in range(df_TEST.shape[0]):
    if df_TEST.loc[i, 'error_pct'] <= .5:
        sum_error = sum_error + 1
print(sum_error, df.shape[0])

## Try to reduce overfitting in RFM

In [ ]:
### #1. Increase number of trees 10-fold
#RF with SPECIFIC libraries removed (and POPU-POPU_LSA removed too).
rfmodel = RandomForestRegressor(n_estimators = 1000, random_state = 42)

X = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#Train model on TRAIN set
rfmodel.fit(X_train, y_train);

#Test model on TEST set (this is the right this to do)
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)
#df of predictions, labels and errors
df_TEST = pd.DataFrame()
df_TEST['Usage_val'] = y_test
df_TEST['Predictions'] = predictions
df_TEST['errors'] = (list(errors))

#Test model on TRAIN set (this is technically the wrong thing to do, but still informative)
predictions = rfmodel.predict(X_train)
errors = abs(predictions - y_train)
#df of predictions, labels and errors
df_TRAIN = pd.DataFrame()
df_TRAIN['Usage_val'] = y_train
df_TRAIN['Predictions'] = predictions
df_TRAIN['errors'] = (list(errors))

#Get feature importance
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#Plot df_TEST Usage vs. Prediction:
# Data
x = df_TEST.iloc[:, 0]
y = df_TEST.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

#Plot df_TRAIN Usage vs. Prediction:
# Data
x = df_TRAIN.iloc[:, 0]
y = df_TRAIN.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))

#Increasing estimators to 1000 did not do anything!
#This is good to know. Took a while to run, so reduce back to 100 trees.

In [ ]:
### #2. Reduce number of trees back to 100. Reduce max_features from auto (total features) to sqrt (sqrt(features))
#RF with SPECIFIC libraries removed (and POPU-POPU_LSA removed too).
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42, max_features = "sqrt")

X = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#Train model on TRAIN set
rfmodel.fit(X_train, y_train);

#Test model on TEST set (this is the right this to do)
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)
#df of predictions, labels and errors
df_TEST = pd.DataFrame()
df_TEST['Usage_val'] = y_test
df_TEST['Predictions'] = predictions
df_TEST['errors'] = (list(errors))

#Test model on TRAIN set (this is technically the wrong thing to do, but still informative)
predictions = rfmodel.predict(X_train)
errors = abs(predictions - y_train)
#df of predictions, labels and errors
df_TRAIN = pd.DataFrame()
df_TRAIN['Usage_val'] = y_train
df_TRAIN['Predictions'] = predictions
df_TRAIN['errors'] = (list(errors))

#Get feature importance
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#Plot df_TEST Usage vs. Prediction:
# Data
x = df_TEST.iloc[:, 0]
y = df_TEST.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

#Plot df_TRAIN Usage vs. Prediction:
# Data
x = df_TRAIN.iloc[:, 0]
y = df_TRAIN.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))

#Doesn't really change much.

In [ ]:
### #3. Reduce number of trees back to 100. Change max_features back to auto (total features). Reduce max_depth to 10 (this is a bit arbitrary and pretty low; can consider increasing).
#RF with SPECIFIC libraries removed (and POPU-POPU_LSA removed too).
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth = 10)

X = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#Train model on TRAIN set
rfmodel.fit(X_train, y_train);

#Test model on TEST set (this is the right this to do)
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)
#df of predictions, labels and errors
df_TEST = pd.DataFrame()
df_TEST['Usage_val'] = y_test
df_TEST['Predictions'] = predictions
df_TEST['errors'] = (list(errors))

#Test model on TRAIN set (this is technically the wrong thing to do, but still informative)
predictions = rfmodel.predict(X_train)
errors = abs(predictions - y_train)
#df of predictions, labels and errors
df_TRAIN = pd.DataFrame()
df_TRAIN['Usage_val'] = y_train
df_TRAIN['Predictions'] = predictions
df_TRAIN['errors'] = (list(errors))

#Get feature importance
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#Plot df_TEST Usage vs. Prediction:
# Data
x = df_TEST.iloc[:, 0]
y = df_TEST.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

#Plot df_TRAIN Usage vs. Prediction:
# Data
x = df_TRAIN.iloc[:, 0]
y = df_TRAIN.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))

#Train set is worse, but test set is no better.

In [ ]:
### #3. Reduce number of trees back to 100. Increase max_depth to 100.
#RF with SPECIFIC libraries removed (and POPU-POPU_LSA removed too).
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth = 100)

X = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#Train model on TRAIN set
rfmodel.fit(X_train, y_train);

#Test model on TEST set (this is the right this to do)
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)
#df of predictions, labels and errors
df_TEST = pd.DataFrame()
df_TEST['Usage_val'] = y_test
df_TEST['Predictions'] = predictions
df_TEST['errors'] = (list(errors))

#Test model on TRAIN set (this is technically the wrong thing to do, but still informative)
predictions = rfmodel.predict(X_train)
errors = abs(predictions - y_train)
#df of predictions, labels and errors
df_TRAIN = pd.DataFrame()
df_TRAIN['Usage_val'] = y_train
df_TRAIN['Predictions'] = predictions
df_TRAIN['errors'] = (list(errors))

#Get feature importance
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#Plot df_TEST Usage vs. Prediction:
# Data
x = df_TEST.iloc[:, 0]
y = df_TEST.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

#Plot df_TRAIN Usage vs. Prediction:
# Data
x = df_TRAIN.iloc[:, 0]
y = df_TRAIN.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))

#Train set score jumps back up, test score still low

In [ ]:
### #4. Try max depth between 10 and 100
#RF with SPECIFIC libraries removed (and POPU-POPU_LSA removed too).
X = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]
    
var_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
for number in var_depth:
    rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth = number)
    rfmodel.fit(X_train, y_train)
    print("max_depth = ", number)
    print("Score for Test set: ", rfmodel.score(X_test, y_test))
    print("Score for Train set: ", rfmodel.score(X_train, y_train))  

In [ ]:
### #5. Try max depth 40, max_features to sqrt (sqrt(features))
#RF with SPECIFIC libraries removed (and POPU-POPU_LSA removed too).
X = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]
    

rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth = 40, max_features = "sqrt")
rfmodel.fit(X_train, y_train)
print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))

#n_estimators = 100
#max_features = 30-50% of the number of features (reducing may reduce overfitting; right now, it's # of features)






In [ ]:
### #6. Try max_features to sqrt (sqrt(features)), min_samples_leaf to 15
#RF with SPECIFIC libraries removed (and POPU-POPU_LSA removed too).
X = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]
    

rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42, max_features = "sqrt", min_samples_leaf = 15)
rfmodel.fit(X_train, y_train)
print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))

In [ ]:
m_s_l_list = [1, 2, 5, 10, 15]
for number in m_s_l_list:
    rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42, max_features = "auto", min_samples_leaf = number)
    rfmodel.fit(X_train, y_train)
    print("min_sample_leaf: ", number)
    print("Score for Test set: ", rfmodel.score(X_test, y_test))
    print("Score for Train set: ", rfmodel.score(X_train, y_train))

In [ ]:
##Back to original model. Is R2 score schewed by a few very bad error?
#Left max_depth at 60, because R2 is about the same as if max_depth not set. 
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth = 60)

X = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#Train model on TRAIN set
rfmodel.fit(X_train, y_train);

#Test model on TEST set (this is the right this to do)
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)
#df of predictions, labels and errors
df_TEST = pd.DataFrame()
df_TEST['Usage_val'] = y_test
df_TEST['Predictions'] = predictions
df_TEST['errors'] = (list(errors))

#Test model on TRAIN set (this is technically the wrong thing to do, but still informative)
predictions = rfmodel.predict(X_train)
errors = abs(predictions - y_train)
#df of predictions, labels and errors
df_TRAIN = pd.DataFrame()
df_TRAIN['Usage_val'] = y_train
df_TRAIN['Predictions'] = predictions
df_TRAIN['errors'] = (list(errors))

#Get feature importance
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#Plot df_TEST Usage vs. Prediction:
# Data
x = df_TEST.iloc[:, 0]
y = df_TEST.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

#Plot df_TRAIN Usage vs. Prediction:
# Data
x = df_TRAIN.iloc[:, 0]
y = df_TRAIN.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))

In [ ]:
#calculate R2 
numerator = sum((df_TEST['Usage_val']-df_TEST['Predictions'])**2)
denominator = sum((df_TEST['Usage_val']-df_TEST['Usage_val'].mean())**2)
1-numerator/denominator

In [ ]:
#TRY to classify instead (0, 1, 2, 3, 4)

In [ ]:
list(df_TEST['errors'])

In [ ]:
print(df_TEST.hist(column='errors'))
print(df_TRAIN.hist(column='errors'))
print(df_TEST.hist(column='Usage_val'))
print(df_TRAIN.hist(column='Usage_val'))

In [ ]:
print(df_TEST[df_TEST['errors'] > 20].head())
len(df_TEST[df_TEST['errors'] > 20])

In [ ]:
#calculate R2 for all where 'Usage_val' < 30.
idx_high = df_TEST[df_TEST['Usage_val'] > 30].index
df_TEST_lowuseonly = df_TEST.drop(idx_high)
numerator = sum((df_TEST_lowuseonly['Usage_val']-df_TEST_lowuseonly['Predictions'])**2)
denominator = sum((df_TEST_lowuseonly['Usage_val']-df_TEST_lowuseonly['Usage_val'].mean())**2)
1-numerator/denominator

In [ ]:
#Save libraries used
my_list = _20PctFSCSList
with open('your_file.txt', 'w') as f:
    for item in my_list:
        f.write("%s\n" % item)

In [ ]:
FSCSKeys_Used_Above_Jan29_10am = _20PctFSCSList

## Are results similar when I pick random set of libraries?

In [ ]:
###GET random set of libraries to remove
Location_Detail_DF_noNaN_noSpace_ri_DropOutliers = Location_Detail_DF_noNaN_noSpace_ri_DropOutliers.reset_index(drop=True)
Concat_w_dummies_noNaN_noSpace_ri_DropOutliers = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.reset_index(drop=True)

UniqueFSCS = set(Location_Detail_DF_noNaN_noSpace_ri_DropOutliers.loc[:, 'FSCSKEY'].tolist())
print(len(UniqueFSCS))

#Pick 20% of FSCSKEYS randomly
Numb_to_sel = int((.20*len(UniqueFSCS)))
_20PctFSCSList = random.sample(UniqueFSCS, Numb_to_sel)

#Get index of these FSCSKEYS from Location_Detail_DF_noNaN_noSpace_ri
Indexlist = list()
for FSCSkey in _20PctFSCSList:
    Indexfor_20PctFSCSList = Location_Detail_DF_noNaN_noSpace_ri_DropOutliers[Location_Detail_DF_noNaN_noSpace_ri_DropOutliers.FSCSKEY == FSCSkey].index
    Indexlist.append(list(Indexfor_20PctFSCSList))
flattened_Indexlist = [val for sublist in Indexlist for val in sublist]
print(len(flattened_Indexlist))

In [ ]:
##Back to original model. Is R2 score schewed by a few very bad error?
#Left max_depth at 60, because R2 is about the same as if max_depth not set. 
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 42, max_depth = 60)

X = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#Train model on TRAIN set
rfmodel.fit(X_train, y_train);

#Test model on TEST set (this is the right this to do)
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)
#df of predictions, labels and errors
df_TEST = pd.DataFrame()
df_TEST['Usage_val'] = y_test
df_TEST['Predictions'] = predictions
df_TEST['errors'] = (list(errors))

#Test model on TRAIN set (this is technically the wrong thing to do, but still informative)
predictions = rfmodel.predict(X_train)
errors = abs(predictions - y_train)
#df of predictions, labels and errors
df_TRAIN = pd.DataFrame()
df_TRAIN['Usage_val'] = y_train
df_TRAIN['Predictions'] = predictions
df_TRAIN['errors'] = (list(errors))

#Get feature importance
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#Plot df_TEST Usage vs. Prediction:
# Data
x = df_TEST.iloc[:, 0]
y = df_TEST.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

#Plot df_TRAIN Usage vs. Prediction:
# Data
x = df_TRAIN.iloc[:, 0]
y = df_TRAIN.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))

In [ ]:
#change random state - does this change predictions a lot?
rfmodel = RandomForestRegressor(n_estimators = 100, random_state = 10, max_depth = 60)

X = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers.drop(columns = ['Usage', 'POPU-POPU_LSA'])
y = Concat_w_dummies_noNaN_noSpace_ri_DropOutliers['Usage']

X_train = X.drop(X.index[flattened_Indexlist])
y_train = y.drop(y.index[flattened_Indexlist])
X_test = X.loc[flattened_Indexlist]
y_test = y.loc[flattened_Indexlist]

#Train model on TRAIN set
rfmodel.fit(X_train, y_train);

#Test model on TEST set (this is the right this to do)
predictions = rfmodel.predict(X_test)
errors = abs(predictions - y_test)
#df of predictions, labels and errors
df_TEST = pd.DataFrame()
df_TEST['Usage_val'] = y_test
df_TEST['Predictions'] = predictions
df_TEST['errors'] = (list(errors))

#Test model on TRAIN set (this is technically the wrong thing to do, but still informative)
predictions = rfmodel.predict(X_train)
errors = abs(predictions - y_train)
#df of predictions, labels and errors
df_TRAIN = pd.DataFrame()
df_TRAIN['Usage_val'] = y_train
df_TRAIN['Predictions'] = predictions
df_TRAIN['errors'] = (list(errors))

#Get feature importance
feature_importances = pd.DataFrame(rfmodel.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

#Plot df_TEST Usage vs. Prediction:
# Data
x = df_TEST.iloc[:, 0]
y = df_TEST.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

#Plot df_TRAIN Usage vs. Prediction:
# Data
x = df_TRAIN.iloc[:, 0]
y = df_TRAIN.iloc[:, 1]
# Fit with polyfit
b, m = polyfit(x, y, 1)
plt.plot(x, y, '.')
plt.plot(x, b + m * x, '-')
plt.show()

print("Score for Test set: ", rfmodel.score(X_test, y_test))
print("Score for Train set: ", rfmodel.score(X_train, y_train))